In [1]:
from pyspark import SparkContext
from pyspark.sql.functions import countDistinct
sc=SparkContext()

22/12/09 18:49:35 WARN Utils: Your hostname, Tobiass-MacBook.local resolves to a loopback address: 127.0.0.1; using 10.27.90.105 instead (on interface en0)
22/12/09 18:49:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/09 18:49:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [46]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, bround
spark = SparkSession.builder \
         .appName('SparkByExamples.com') \
         .getOrCreate()

Pseude Code:

agrupar por user, device, gt
    para x, y, z sacar
        median
        desv
        max
        min

join data from different datasets into RDDs
Analize RDDs

Don´t repeat code!
avoid many (unnecessary) lines of code / repetitions

In [4]:
columns = ["index", "arrival_time", "creation_time", "x", "y", "z", "user", "model", "device", "activity"]

p_acc = sc.textFile('Phones_accelerometer.csv')
pa = p_acc.map(lambda line: [x for x in line.split(',')]).toDF(columns)
p_gyro = sc.textFile('Phones_gyroscope.csv')
pg = p_gyro.map(lambda line: [x for x in line.split(',')]).toDF(columns)
w_acc = sc.textFile('Watch_accelerometer.csv')
wa = w_acc.map(lambda line: [x for x in line.split(',')]).toDF(columns)
w_gyro = sc.textFile('Watch_gyroscope.csv')
wg = w_gyro.map(lambda line: [x for x in line.split(',')]).toDF(columns)


print(pg.take(5))

[Row(index='0', arrival_time='1424696633909', creation_time='1424696631914042029', x='0.013748169', y='-0.0006256103500000001', z='-0.023376465', user='a', model='nexus4', device='nexus4_1', activity='stand'), Row(index='1', arrival_time='1424696633909', creation_time='1424696631919046912', x='0.014816283999999999', y='-0.0016937256', z='-0.02230835', user='a', model='nexus4', device='nexus4_1', activity='stand'), Row(index='2', arrival_time='1424696633918', creation_time='1424696631924051794', x='0.0158844', y='-0.0016937256', z='-0.021240234', user='a', model='nexus4', device='nexus4_1', activity='stand'), Row(index='3', arrival_time='1424696633919', creation_time='1424696631929117712', x='0.016952515', y='-0.003829956', z='-0.02017212', user='a', model='nexus4', device='nexus4_1', activity='stand'), Row(index='4', arrival_time='1424696633928', creation_time='1424696631934214148', x='0.0158844', y='-0.007034301800000001', z='-0.02017212', user='a', model='nexus4', device='nexus4_1', 

In [5]:
pa.printSchema()
pa.show(truncate=False)

root
 |-- index: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- creation_time: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- z: string (nullable = true)
 |-- user: string (nullable = true)
 |-- model: string (nullable = true)
 |-- device: string (nullable = true)
 |-- activity: string (nullable = true)

+-----+-------------+-------------------+-------------------+------------------+-----------------+----+------+--------+--------+
|index|arrival_time |creation_time      |x                  |y                 |z                |user|model |device  |activity|
+-----+-------------+-------------------+-------------------+------------------+-----------------+----+------+--------+--------+
|0    |1424696633908|1424696631913248572|-5.958191          |0.6880646         |8.135345         |a   |nexus4|nexus4_1|stand   |
|1    |1424696633909|1424696631918283972|-5.95224           |0.6702118         |8.136536         |a   

In [7]:
dist_pa = pa.dropDuplicates(["user"]).select("user", "model", "device", "activity")
dist_pa.show(truncate=False)

#print(df.dropDuplicates(["salary"]).select("salary").collect())

+----+-----+------+--------+
|user|model|device|activity|
+----+-----+------+--------+
|a   |gear |gear_1|stand   |
+----+-----+------+--------+



In [33]:
to_string = ["x", "y", "z"]

In [35]:
pa_xyz = pa.drop("index", "arrival_time", "creation_time", "device")
for column in to_string:
    pa_xyz = pa_xyz.withColumn(column, col(column).cast('float'))
pa_xyz.show()


+----------+----------+---------+----+------+--------+
|         x|         y|        z|user| model|activity|
+----------+----------+---------+----+------+--------+
| -5.958191| 0.6880646| 8.135345|   a|nexus4|   stand|
|  -5.95224| 0.6702118| 8.136536|   a|nexus4|   stand|
|-5.9950867| 0.6535492| 8.204376|   a|nexus4|   stand|
|-5.9427185| 0.6761627| 8.128204|   a|nexus4|   stand|
| -5.991516|0.64164734| 8.135345|   a|nexus4|   stand|
| -5.965332| 0.6297455| 8.128204|   a|nexus4|   stand|
| -5.991516| 0.6356964|  8.16272|   a|nexus4|   stand|
| -5.915344|0.63093567| 8.105591|   a|nexus4|   stand|
| -5.984375| 0.6940155| 8.067505|   a|nexus4|   stand|
| -5.937958|0.71543884| 8.090118|   a|nexus4|   stand|
| -5.902252| 0.6678314| 8.069885|   a|nexus4|   stand|
|-5.9498596|0.68092346| 8.119873|   a|nexus4|   stand|
|-5.9796143| 0.7416229|8.0841675|   a|nexus4|   stand|
|-5.9617615|0.71424866| 8.155579|   a|nexus4|   stand|
|  -5.95343| 0.7130585| 8.153198|   a|nexus4|   stand|
|-5.866546

In [18]:
key = ["user", "model", "activity"]

In [36]:

avg = pa_xyz.groupBy(key).agg({"x":"avg", "y":"avg", "z":"avg"})
avg.show()

+----+------+--------+------------------+------------------+----------------+
|user| model|activity|            avg(x)|            avg(y)|          avg(z)|
+----+------+--------+------------------+------------------+----------------+
|   a|nexus4|   stand|-6.026499951171875|0.9334959503173829|8.01364601135254|
+----+------+--------+------------------+------------------+----------------+



In [37]:
std = pa_xyz.groupBy(key).agg({"x":"stddev", "y":"stddev", "z":"stddev"})
std.show()

+----+------+--------+-------------------+-------------------+------------------+
|user| model|activity|          stddev(x)|          stddev(y)|         stddev(z)|
+----+------+--------+-------------------+-------------------+------------------+
|   a|nexus4|   stand|0.18456097451689352|0.24044618117892705|0.1760086615510722|
+----+------+--------+-------------------+-------------------+------------------+



In [40]:
max = pa_xyz.groupBy(key).agg({"x":"max", "y":"max", "z":"max"})
max.show()

+----+------+--------+----------+---------+--------+
|user| model|activity|    max(x)|   max(y)|  max(z)|
+----+------+--------+----------+---------+--------+
|   a|nexus4|   stand|-5.5202026|1.9472809|8.638794|
+----+------+--------+----------+---------+--------+



In [42]:
min = pa_xyz.groupBy(key).agg({"x":"min", "y":"min", "z":"min"})
min.show()

+----+------+--------+----------+-----------+--------+
|user| model|activity|    min(x)|     min(y)|  min(z)|
+----+------+--------+----------+-----------+--------+
|   a|nexus4|   stand|-7.0448303|-0.84251404|7.149872|
+----+------+--------+----------+-----------+--------+



In [49]:
out = avg.join(std,key).join(max, key).join(min, key)
out.show()

+----+------+--------+------------------+------------------+----------------+-------------------+-------------------+------------------+----------+---------+--------+----------+-----------+--------+
|user| model|activity|            avg(x)|            avg(y)|          avg(z)|          stddev(x)|          stddev(y)|         stddev(z)|    max(x)|   max(y)|  max(z)|    min(x)|     min(y)|  min(z)|
+----+------+--------+------------------+------------------+----------------+-------------------+-------------------+------------------+----------+---------+--------+----------+-----------+--------+
|   a|nexus4|   stand|-6.026499951171875|0.9334959503173829|8.01364601135254|0.18456097451689352|0.24044618117892705|0.1760086615510722|-5.5202026|1.9472809|8.638794|-7.0448303|-0.84251404|7.149872|
+----+------+--------+------------------+------------------+----------------+-------------------+-------------------+------------------+----------+---------+--------+----------+-----------+--------+

22/1

In [53]:
end = pa_xyz.groupBy(key).agg({"x":"avg", "y":"avg", "z":"avg", "x":"stddev", "y":"stddev", "z":"stddev", "x":"max", "y":"max", "z":"max", "x":"min", "y":"min", "z":"min"})
end.show()

+----+------+--------+----------+-----------+--------+
|user| model|activity|    min(x)|     min(y)|  min(z)|
+----+------+--------+----------+-----------+--------+
|   a|nexus4|   stand|-7.0448303|-0.84251404|7.149872|
+----+------+--------+----------+-----------+--------+

